In [4]:
import pandas as pd

In [5]:
raw_data = pd.read_csv('/Volumes/data/stuff/IRS_zipcode.csv')

In [6]:
raw_data.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,815440,477700,105350,221200,440830,1296920,...,367320,330066,0,0,0,0,63420,51444,711580,1831661
1,1,AL,0,2,495830,211930,142340,128890,272440,996240,...,398050,984297,0,0,0,0,74090,110889,416090,1173463
2,1,AL,0,3,263390,83420,137870,36340,154880,584000,...,253180,1349246,0,0,0,0,64000,143060,195130,543284
3,1,AL,0,4,167190,29420,124060,10610,99700,421720,...,165830,1425430,0,0,0,0,45020,128920,117410,381329
4,1,AL,0,5,217440,20240,188080,4880,129410,601040,...,216720,3922449,390,155,60,19,82940,423629,126130,506526


Remove any values with 0 or 99999

In [3]:
no_zeroes = raw_data.loc[
            (raw_data['zipcode'] != 0) & (raw_data['zipcode'] != 99999)]

Remove any zip codes with less than 4 digits

In [4]:
all_five = no_zeroes.loc[no_zeroes['zipcode'].apply(lambda x: len(str(x)) == 5)]

Pivot the data to remove redundant columns

In [5]:
pivoted = pd.pivot_table(data=all_five,aggfunc='sum',values='N1',columns='agi_stub',index=['zipcode']).unstack('agi_stub')

In [6]:
pivoted = pivoted.unstack().T.reset_index().rename_axis(None,axis=1)

In [7]:
pivoted.head()

,zipcode,1,2,3,4,5,6
0,10001,3740.0,2500.0,1950.0,1410.0,2580.0,2340.0
1,10002,21180.0,8210.0,4810.0,2710.0,3620.0,1650.0
2,10003,5030.0,3870.0,3960.0,3220.0,6300.0,6280.0
3,10004,360.0,250.0,290.0,210.0,520.0,850.0
4,10005,690.0,750.0,860.0,660.0,1360.0,1620.0


Add the State

In [8]:
states = no_zeroes[['STATE','zipcode']]

In [9]:
states.drop_duplicates(['STATE','zipcode'],inplace=True)

/Users/debrastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
states.head()

,STATE,zipcode
6,AL,35004
12,AL,35005
18,AL,35006
24,AL,35007
30,AL,35010


In [11]:
pivoted = pd.merge(states,pivoted,on='zipcode')

In [12]:
pivoted.head()

,STATE,zipcode,1,2,3,4,5,6
0,AL,35004,1510.0,1410.0,950.0,650.0,630.0,60.0
1,AL,35005,1310.0,960.0,450.0,200.0,180.0,0.0
2,AL,35006,430.0,330.0,190.0,120.0,130.0,0.0
3,AL,35007,4070.0,2650.0,1820.0,1340.0,1820.0,240.0
4,AL,35010,3620.0,2070.0,890.0,530.0,620.0,180.0


In [13]:
pivoted.to_csv('/Volumes/data/stuff/IRS_zipcodes_clean.csv',index=False)